<a href="https://colab.research.google.com/github/Hamedsamak66/Bitcoin-Price-Prediction/blob/main/BTC-new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install pmdarima
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
import matplotlib.pyplot as plt

In [48]:
# Step 1: Read and preprocess the dataset
df = pd.read_csv('/content/sample_data/Dataset.csv')
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

In [49]:
# Identify numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()


In [50]:
# Ensure 'BTC' is included in numeric_cols if it was excluded for some reason
if 'BTC' not in numeric_cols:
    numeric_cols.append('BTC')

In [51]:
# Scale only the numeric columns
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[numeric_cols])

In [52]:
# Create sequences for LSTM
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length - 1):
        X.append(data[i:(i + seq_length), :])
        y.append(data[i + seq_length, -1])
    return np.array(X), np.array(y)

seq_length = 60  # Use 60 days of data to predict the next day's BTC price
X, y = create_sequences(scaled_data, seq_length)

In [53]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Step 2: Build and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50
46/46 [==============================] - 56s 86ms/step - loss: 0.0151 - val_loss: 0.0047
Epoch 2/50
46/46 [==============================] - 3s 58ms/step - loss: 0.0055 - val_loss: 0.0026
Epoch 3/50
46/46 [==============================] - 3s 56ms/step - loss: 0.0049 - val_loss: 0.0020
Epoch 4/50
46/46 [==============================] - 3s 69ms/step - loss: 0.0036 - val_loss: 0.0017
Epoch 5/50
46/46 [==============================] - 3s 74ms/step - loss: 0.0042 - val_loss: 0.0019
Epoch 6/50
46/46 [==============================] - 3s 61ms/step - loss: 0.0034 - val_loss: 0.0011
Epoch 7/50
46/46 [==============================] - 3s 58ms/step - loss: 0.0030 - val_loss: 9.1908e-04
Epoch 8/50
46/46 [==============================] - 3s 55ms/step - loss: 0.0028 - val_loss: 9.4844e-04
Epoch 9/50
46/46 [==============================] - 4s 84ms/step - loss: 0.0026 - val_loss: 0.0012
Epoch 10/50
46/46 [==============================] - 3s 57ms/step - loss: 0.0027 - val_loss: 0.0014


In [ ]:
# Step 3: Evaluate the LSTM model
y_pred_lstm = model.predict(X_test)

# Inverse transform the predictions and actual values
y_pred_lstm = scaler.inverse_transform(np.concatenate((np.zeros((y_pred_lstm.shape[0], len(numeric_cols) - 1)), y_pred_lstm), axis=1))[:, -1]
y_test_lstm = scaler.inverse_transform(np.concatenate((np.zeros((y_test.shape[0], len(numeric_cols) - 1)), y_test.reshape(-1, 1)), axis=1))[:, -1]

# Calculate R-squared and MSE for LSTM model
r2_lstm = r2_score(y_test_lstm, y_pred_lstm)
mse_lstm = mean_squared_error(y_test_lstm, y_pred_lstm)

print(f'LSTM Model - R-squared: {r2_lstm}, MSE: {mse_lstm}')

In [ ]:
# Step 4: Build and train the Linear Regression model
# Reshape data for Linear Regression
X_lr = scaled_data[:, :-1]  # Use all columns except the last one (BTC price) for features
y_lr = scaled_data[:, -1]   # Use BTC price as the target

# Split the data into training and testing sets
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_lr, y_lr, test_size=0.2, random_state=42)

# Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_lr, y_train_lr)

In [ ]:
# Step 5: Evaluate the Linear Regression model
y_pred_lr = lr_model.predict(X_test_lr)

# Inverse transform the predictions and actual values
y_pred_lr = scaler.inverse_transform(np.concatenate((np.zeros((y_pred_lr.shape[0], len(numeric_cols) - 1)), y_pred_lr.reshape(-1, 1)), axis=1))[:, -1]
y_test_lr = scaler.inverse_transform(np.concatenate((np.zeros((y_test_lr.shape[0], len(numeric_cols) - 1)), y_test_lr.reshape(-1, 1)), axis=1))[:, -1]

# Calculate R-squared and MSE for Linear Regression model
r2_lr = r2_score(y_test_lr, y_pred_lr)
mse_lr = mean_squared_error(y_test_lr, y_pred_lr)

print(f'Linear Regression Model - R-squared: {r2_lr}, MSE: {mse_lr}')


In [ ]:
# Step 6: Build and train the ARIMA model using auto_arima
# ARIMA model works on univariate time series, use only the BTC price column
btc_prices = df['BTC'].values

# Split the data into training and testing sets
train_size = int(len(btc_prices) * 0.8)
train_btc, test_btc = btc_prices[:train_size], btc_prices[train_size:]

# Use auto_arima to find the best parameters for ARIMA
arima_model = auto_arima(train_btc, seasonal=False, stepwise=True, suppress_warnings=True)
arima_order = arima_model.order
print(f'Best ARIMA order: {arima_order}')

# Train the ARIMA model with the best order found
arima_model = ARIMA(train_btc, order=arima_order)
arima_model_fit = arima_model.fit()



In [ ]:

# Step 7: Evaluate the ARIMA model
pred_start = len(train_btc)
pred_end = len(btc_prices) - 1
y_pred_arima = arima_model_fit.predict(start=pred_start, end=pred_end, dynamic=False)

# Calculate R-squared and MSE for ARIMA model
r2_arima = r2_score(test_btc, y_pred_arima)
mse_arima = mean_squared_error(test_btc, y_pred_arima)

print(f'ARIMA Model - R-squared: {r2_arima}, MSE: {mse_arima}')


In [ ]:
# Step 8: Plot the R-squared values for comparison
models = ['LSTM', 'Linear Regression', 'ARIMA']
r2_values = [r2_lstm, r2_lr, r2_arima]

plt.figure(figsize=(10, 6))
plt.bar(models, r2_values, color=['blue', 'green', 'red'])
plt.xlabel('Model')
plt.ylabel('R-squared')
plt.title('Comparison of R-squared values for different models')
plt.ylim(0, 1)
plt.show()